ML DL
Projet 3 -Audio detection-
Master IT 2024 - 2025


In [ ]:
import kagglehub
bigguyubuntu_quran_ayat_speech_to_text_path = kagglehub.dataset_download('bigguyubuntu/quran-ayat-speech-to-text')

print('Data source import complete.')


100%|██████████| 39.1G/39.1G [31:33<00:00, 22.2MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install pydub

## Read Dataset

In [ ]:
import os
import joblib
import torch
import torchaudio
import pandas as pd
import numpy as np
import librosa
import matplotlib.pyplot as plt
import seaborn as sns
import torchaudio.transforms as T
import tensorflow as tf

from tqdm import tqdm
from pydub import AudioSegment
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, ConfusionMatrixDisplay

In [ ]:
folders = [
    ("AbdulSamad_64kbps", "abdulsamad"),
    ("Abdul_Basit_Murattal_64kbps", "abdulbasit"),
    ("Abdullaah_3awwaad_Al-Juhaynee_128kbps", "abdallah_3waad"),
    ("Abdullah_Basfar_32kbps", "abdallah_basfar"),
    ("Abdurrahmaan_As-Sudais_64kbps", "sudais"),
    ("Abu_Bakr_Ash-Shaatree_64kbps", "shatree"),
    ("Alafasy_64kbps", "afasy"),
    ("Ali_Jaber_64kbps", "ali_jaber"),
    ("Ayman_Sowaid_64kbps", "ayman_sowaid"),
    ("Banna_32kbps", "banna"),
    ("Fares_Abbad_64kbps", "fares_abbad"),
    ("Ghamadi_40kbps", "ghamadi"),
    ("Hani_Rifai_192kbps", "hani_rifai"),
    ("Hudhaify_64kbps", "hudhaify"),
    ("Husary_64kbps", "husary"),
    ("Ibrahim_Akhdar_32kbps", "ibrahim_akhdar"),
    ("Maher_AlMuaiqly_64kbps", "maher_almuaiqly"),
    ("Minshawy_Murattal_128kbps", "minshawy"),
    ("Mohammad_al_Tablaway_64kbps", "tablawy"),
    ("Mostafa_Ismail_128kbps", "mostafa_ismail"),
    ("Muhammad_Ayyoub_64kbps", "mohammed_ayyoub"),
    ("Muhammad_Jibreel_64kbps", "mohammed_jibreel"),
    ("Muhsin_Al_Qasim_192kbps", "muhsin_alqasim"),
    ("Saood_ash-Shuraym_64kbps", "soaad"),
    ("Yaser_Salamah_128kbps", "yaser_salamah"),
    ("Yasser_Ad-Dussary_128kbps", "yasser_addussary"),
]

In [ ]:
len(folders)

26

## Extract Features

In [ ]:
X = []
y = []

In [ ]:
def extract_mfcc_features(file_path):
    try:
        waveform, sample_rate = torchaudio.load(file_path)
        mfccs = torchaudio.compliance.kaldi.mfcc(waveform=waveform.cuda(), sample_frequency=sample_rate)
        return mfccs.mean(dim=0).cpu().numpy()
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

In [ ]:
print(f'Downloaded dataset path: {bigguyubuntu_quran_ayat_speech_to_text_path}')

Downloaded dataset path: /root/.cache/kagglehub/datasets/bigguyubuntu/quran-ayat-speech-to-text/versions/1


In [ ]:
for folder, reader in folders:
    # Construct the full path by including the correct subdirectory structure
    reader_folder_path = os.path.join(bigguyubuntu_quran_ayat_speech_to_text_path, 'Quran_Ayat_public', 'audio_data', folder)

    # Check if the directory exists before listing its contents
    if os.path.exists(reader_folder_path):
        audio_list = os.listdir(reader_folder_path)
        print(f'Processing {folder} ({len(audio_list)} fichiers)')

        for audio in tqdm(audio_list):
            audio_path = os.path.join(reader_folder_path, audio)
            features = extract_mfcc_features(audio_path)

            if features is not None:
                X.append(features)
                y.append(reader)
    else:
        print(f"Warning: Directory not found: {reader_folder_path}")

Processing AbdulSamad_64kbps (6236 fichiers)


100%|██████████| 6236/6236 [06:07<00:00, 16.99it/s]


Processing Abdul_Basit_Murattal_64kbps (6236 fichiers)


100%|██████████| 6236/6236 [02:42<00:00, 38.43it/s]


Processing Abdullaah_3awwaad_Al-Juhaynee_128kbps (6236 fichiers)


...

100%|██████████| 6236/6236 [01:41<00:00, 61.66it/s]


Processing Yaser_Salamah_128kbps (6204 fichiers)


100%|██████████| 6204/6204 [03:08<00:00, 33.00it/s]


Processing Yasser_Ad-Dussary_128kbps (6236 fichiers)


100%|██████████| 6236/6236 [03:09<00:00, 32.96it/s]


In [ ]:
print(len(X), len(y))

162101 162101


### Save as pandas DF

In [ ]:
df = pd.DataFrame(
    y,
    columns=["Class"]
)

In [ ]:
features_df = pd.DataFrame(X)
df = pd.concat([df['Class'], features_df], axis=1)

In [ ]:
df.head()

,Class,0,1,2,3,4,5,6,7,8,9,10,11,12
0,abdulsamad,-18.020639,19.120899,-71.996880,48.498222,-54.396507,41.037891,-26.674059,-24.906404,33.288929,-42.214642,-9.683245,-35.587917,13.058946
1,abdulsamad,-16.568369,19.347542,-71.439941,51.148735,-59.063435,50.086010,-23.885296,-23.842672,31.205585,-39.813011,-1.794181,-35.441864,10.532916
2,abdulsamad,-12.861549,27.008080,-71.368736,46.507347,-60.948681,30.004642,-26.057545,-23.226192,33.532871,-36.001801,-1.250048,-25.841499,16.419941
3,abdulsamad,-19.844347,19.432442,-67.854553,44.119602,-53.010902,37.928493,-27.774044,-21.805658,27.421648,-32.047119,-1.608690,-35.705479,13.375690
4,abdulsamad,-16.130190,19.921894,-77.283463,49.980156,-59.733593,40.002346,-21.664104,-31.277723,35.202286,-39.017677,-7.554701,-36.969288,7.713711


### Save DF

In [ ]:
df.to_csv('./quran-readers-audio-processed.csv')

### Label Encoder

In [ ]:
le = LabelEncoder()

In [ ]:
y_transformed = le.fit_transform(y)

In [ ]:
y_transformed

array([ 3,  3,  3, ..., 25, 25, 25])

## Implement Gaussian Naive Bayes Classifier using TensorFlow

Given set of features $x_1, x_2,\dots, x_{13}$ as $X_i$,
and set of classes $c_1, c_2,\dots, c_{26}$ as $C_k$ We want to build a model that find the value of $P(C_k\mid X_i)$. Then taking the class with maximum probability to find the best estimated class.

$P(C_k\mid X_i) = \frac{P(X_i\mid C_k)\cdot P(C_k)}{P(X_i)}$

By definition of conditional probability, the numerator is just the joint probability distribution $P(X_i\mid C_k)\cdot P(C_k)$, and can be factored using the chain rule:

$P(C_k, X_i) = P(x_1|x_2,\dots,x_{13},C_k)\cdot{P(x_2|x_3,\dots,x_{13},C_k)\cdot{P(x_i\mid{C_k})\cdot{P(C_k)}}}$

For simplicity, we can assume that each feature depends only on the class:

$P(C_k, X_i) = P(x_1,\mid{C_k})\cdot{P(x_2,\mid{C_k})\cdot{P(x_i,\mid{C_k})\cdot{P(C_k)}}}$

we omitted the denominator since it is the same across all classes, i.e., acts merely as a scaling factor:

$P(C_k\mid X_i) = P(X_i\mid C_k)\cdot P(C_k)$

$Y_{predicted} = \arg\max P(X_i\mid C_k)\cdot P(C_k)$

The estimation of $P(C_k)$ is straightforward, we just compute each class’s relative frequency in the training set:

$P(C_k) = \frac{\text{number samples of class } C_k}{\text{total number of samples}}$

We can remove $P(C_k)$ out of $\arg\max$ since it doesn't depend on $X_i$:

$Y_{predicted} = P(C_k)\cdot\arg\max \prod^{n}_{i=1}{P(X_i\mid C_k)}$

Taking $\log$ to avoid underflow problems:

$Y_{predicted} = \log{(P(C_k)\cdot\arg\max \prod^{n}_{i=1}{P(X_i\mid C_k)})}$

$Y_{predicted} = \arg\max{(\sum^{n}_{i=1}{\log{P(X_i\mid C_k)}})} + \log{P(C_k)}$

Assuming that $P(X_i\mid{C_k})$ comes from an iid Gaussian Distribution

$P(X_i\mid{C_k}) = \frac{1}{\sqrt{2 \pi \sigma_k^2}} \exp\left(-\frac{(X_i - \mu_k)^2}{2\sigma_k^2}\right)$

$Y_{\text{predicted}} = \arg\max \left( \sum_{i=1}^{n} \log \left( \frac{1}{\sqrt{2 \pi \sigma_k^2}} \exp\left(-\frac{(X_i - \mu_k)^2}{2\sigma_k^2}\right) \right) \right) + \log P(C_k)
$

Finally :

$\mu_k = \frac{1}{n} \sum_{i=1}^{n} X_i$

$\sigma_k^2 = \frac{1}{n} \sum_{i=1}^{n} (X_i - \mu_k)^2$

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
class GaussianNaiveBayes(tf.Module):
    def __init__(self):
        super().__init__()

        self.classes = None
        self.mean = None
        self.var = None
        self.prior = None

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32), tf.TensorSpec(shape=[None], dtype=tf.int64)])
    def fit(self, X, y):
        X = X.numpy()
        y = y.numpy()

        self.classes = np.unique(y)

        self.mean = np.zeros((len(self.classes), X.shape[1]), dtype=np.float32)
        self.var = np.zeros((len(self.classes), X.shape[1]), dtype=np.float32)
        self.prior = np.zeros(len(self.classes), dtype=np.float32)

        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = np.mean(X_c, axis=0)
            self.var[idx, :] = np.var(X_c, axis=0)
            self.prior[idx] = X_c.shape[0] / float(X.shape[0])

    @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
    def predict(self, X):
        def _calculate_likelihood(class_idx, X):
            mean = self.mean[class_idx]
            var = self.var[class_idx]
            numerator = tf.exp(-(X - mean) ** 2 / (2 * var))
            denominator = tf.sqrt(2 * np.pi * var)
            return numerator / denominator

        def _calculate_posterior(X):
            posteriors = []
            for idx in range(len(self.classes)):
                prior = tf.math.log(self.prior[idx])
                likelihood = tf.reduce_sum(tf.math.log(_calculate_likelihood(idx, X)), axis=1)
                posterior = prior + likelihood
                posteriors.append(posterior)
            return tf.transpose(tf.convert_to_tensor(posteriors))

        posteriors = _calculate_posterior(X)
        predictions = tf.argmax(posteriors, axis=1)
        return predictions

In [ ]:
gnb = GaussianNaiveBayes()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Fit the Gaussian Naive Bayes model using the training data
gnb.fit(tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train))

In [ ]:
gnb.fit(tf.convert_to_tensor(X_train), tf.convert_to_tensor(y_train))

In [ ]:
y_pred = gnb.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {precision * 100:.2f}%')

Accuracy: 91.93%
Precision: 92.23%
Recall: 92.23%


Save model

In [ ]:
converter = tf.lite.TFLiteConverter.from_concrete_functions([gnb.predict.get_concrete_function()])
tflite_model = converter.convert()

In [ ]:
with open('gnb_model.tflite', 'wb') as f:
    f.write(tflite_model)

print("Model successfully converted to TFLite!")

Model successfully converted to TFLite!


In [ ]:
# -------------------------------------------
# Test d'un fichier audio avec le modèle ML
# -------------------------------------------

from pydub import AudioSegment
import numpy as np

# Charger le fichier test.wav (s'assurer qu'il est mono et en 16kHz)
# Assurez-vous que "1_16.wav" existe dans le répertoire courant ou spécifiez le chemin complet
audio = AudioSegment.from_wav("1_16.wav")
audio = audio.set_channels(1).set_frame_rate(16000)
audio.export("test_converted.wav", format="wav")

# Extraire les caractéristiques (MFCC)
# La fonction extract_mfcc_features doit être définie dans une cellule précédente
features = extract_mfcc_features("test_converted.wav")
features = np.array(features).reshape(1, -1)  # Reshape pour correspondre à l’entrée du modèle

# Prédiction
# Use the correct variable name 'gnb' instead of 'model'
prediction = gnb.predict(features)

# Afficher la prédiction
# If you want the original class name, you'll need to inverse transform the prediction
# Assuming 'le' (LabelEncoder) is available from previous cells
predicted_class_index = prediction[0].numpy() # Convert TensorFlow tensor to numpy
predicted_class_name = le.inverse_transform([predicted_class_index])[0] # Inverse transform the predicted index

print(f"Classe prédite (index) : {predicted_class_index}")
print(f"Classe prédite (nom) : {predicted_class_name}")

Classe prédite (index) : 6
Classe prédite (nom) : ayman_sowaid


In [ ]:
import joblib

# Sauvegarde du modèle
joblib.dump(gnb, "model_gnb.pkl")

# Sauvegarde du label encoder
joblib.dump(le, "label_encoder.pkl")


['label_encoder.pkl']

In [ ]:
from google.colab import files

files.download("model_gnb.pkl")
files.download("label_encoder.pkl")
files.download("gnb_model.tflite")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>